In [ ]:
import time
import pickle
import warnings
import gc
import copy
import numpy as np
import torch
import torch.nn as nn
import torchaudio
from tqdm import tqdm, tqdm_notebook
from torch.utils.data import Dataset, DataLoader
from matplotlib import colors, pyplot as plt
from IPython.display import clear_output
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os

In [ ]:
# Define your custom dataset class
def signal2pytorch(x):
    if len(x.shape)==1: #mono:
        X = np.expand_dims(X, axis=0)  #add batch dimension (here only 1 batch)
    X=torch.from_numpy(X)
    X=X.type(torch.Tensor)
    X=X.permute(1,0,2)  #make batch dimension first
    return X

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/DL_Project/Clean /content/clean ./

!cp -r /content/drive/MyDrive/DL_Project/Noisy  /content/noisy ./

cp: cannot stat '/content/drive/MyDrive/DL_Project/Clean': No such file or directory
cp: cannot stat '/content/clean': No such file or directory
cp: cannot stat '/content/drive/MyDrive/DL_Project/Noisy': No such file or directory
cp: cannot stat '/content/noisy': No such file or directory


In [ ]:
from pathlib import Path

In [ ]:
SAMPLE_RATE = 48000
N_FFT = (SAMPLE_RATE * 64) // 1000
HOP_LENGTH = (SAMPLE_RATE * 16) // 1000

In [ ]:
class SpeechDataset(Dataset):

    def __init__(self, noisy_files, clean_files):
        super().__init__()
        # list of files
        self.noisy_files = sorted(noisy_files)
        self.clean_files = sorted(clean_files)

        # stft parameters
        self.n_fft = n_fft
        self.hop_length = hop_length

        self.len_ = len(self.noisy_files)

        # fixed len
        self.max_len = 165000


    def __len__(self):
        return self.len_

    def load_sample(self, file):
        waveform, _ = torchaudio.load(file)
        return waveform

    def __getitem__(self, index):
        # load to tensors and normalization
        clean_audio = self.load_sample(self.clean_files[index])
        noisy_audio = self.load_sample(self.noisy_files[index])
        print("Shape of noisy audio ",noisy_audio.shape)
        print(noisy_audio)
        print("Shape of clean audio ",clean_audio.shape)
        print(clean_audio)
        clean_audio = self._prepare_sample(clean_audio)
        noisy_audio = self._prepare_sample(noisy_audio)
        print("Shape of noisy audio AT ",noisy_audio.shape)
        print(noisy_audio)
        print("Shape of clean audio AT",clean_audio.shape)
        print(clean_audio)
        clean_audio_norm = clean_audio / np.abs(clean_audio.max())
        noisy_audio_norm = noisy_audio / np.abs(noisy_audio.max())
        clean_audio = signal2pytorch(clean_audio_norm).to(device)
        noisy_audio = signal2pytorch(noisy_audio_norm).to(device)
        desired_shape = (1, 1, 90000)
        clean_audio_truncated = clean_audio[:, :, :desired_shape[2]]
        noisy_audio_truncated = noisy_audio[:, :, :desired_shape[2]]
        return noisy_audio_truncated, clean_audio_truncated

    def _prepare_sample(self, waveform):
        waveform = waveform.numpy()
        current_len = waveform.shape[1]
        output = np.zeros((1, self.max_len), dtype='float32')
        output[0, -current_len:] = waveform[0, :self.max_len]
        output = torch.from_numpy(output)

        return output

In [ ]:
TRAIN_INPUT_DIR = Path('/content/noisy_trainset_wav')
TRAIN_TARGET_DIR = Path('/content/clean_trainset_wav')


TEST_INPUT_DIR = Path('/content/noisy_testset_wav')
TEST_TARGET_DIR = Path('/content/clean_testset_wav')

train_input_files = sorted(list(TRAIN_INPUT_DIR.rglob('*.wav')))
train_target_files = sorted(list(TRAIN_TARGET_DIR.rglob('*.wav')))
print(train_input_files)
print(train_target_files)


test_input_files = sorted(list(TEST_INPUT_DIR.rglob('*.wav')))
test_target_files = sorted(list(TEST_TARGET_DIR.rglob('*.wav')))

print("No. of Training files:",len(train_input_files))
# print("No. of Testing files:",len(test_noisy_files))
train_dataset = SpeechDataset(train_input_files, train_target_files)
train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)

test_dataset = SpeechDataset(test_input_files, test_target_files)
test_loader = DataLoader(test_dataset, batch_size=30, shuffle=True)

[PosixPath('/content/Noisy/noisy_p232_001_audio_chunk_0.wav'), PosixPath('/content/Noisy/noisy_p232_002_audio_chunk_0.wav'), PosixPath('/content/Noisy/noisy_p232_002_audio_chunk_1.wav'), PosixPath('/content/Noisy/noisy_p232_003_audio_chunk_0.wav'), PosixPath('/content/Noisy/noisy_p232_003_audio_chunk_1.wav'), PosixPath('/content/Noisy/noisy_p232_003_audio_chunk_2.wav'), PosixPath('/content/Noisy/noisy_p232_003_audio_chunk_3.wav'), PosixPath('/content/Noisy/noisy_p232_005_audio_chunk_0.wav'), PosixPath('/content/Noisy/noisy_p232_005_audio_chunk_1.wav'), PosixPath('/content/Noisy/noisy_p232_005_audio_chunk_2.wav'), PosixPath('/content/Noisy/noisy_p232_005_audio_chunk_3.wav')]
[PosixPath('/content/Clean/p232_001_audio_chunk_0.wav'), PosixPath('/content/Clean/p232_002_audio_chunk_0.wav'), PosixPath('/content/Clean/p232_002_audio_chunk_1.wav'), PosixPath('/content/Clean/p232_003_audio_chunk_0.wav'), PosixPath('/content/Clean/p232_003_audio_chunk_1.wav'), PosixPath('/content/Clean/p232_003_a

In [1]:
# for batch_idx, (noisy_audio, clean_audio) in enumerate(train_loader):
#   print("NS",noisy_audio.shape)
#   print("CS",clean_audio.shape)
#   temp_audio = noisy_audio[0]
#   temp_audio=np.array(temp_audio.cpu())
#   xrek_noisy=temp_audio[:,0,:]
#   print("xrek_noisy ",xrek_noisy)
#   display(ipd.Audio(xrek_noisy, rate=48000));
#   xrek_clean=clean_audio[0][:,0,:]
#   print("xrek_clean ",xrek_clean)
#   display(ipd.Audio(xrek_clean.cpu(), rate=48000));


In [ ]:
class Convautoenc(nn.Module):
    def __init__(self):
        super(Convautoenc, self).__init__()
        #Analysis Filterbank with downsampling of N=1024, filter length of 2N, but only N/2 outputs:
        self.conv1=nn.Conv1d(in_channels=1, out_channels=32, kernel_size=2048, stride=1024, padding=1023, bias=True) #Padding for 'same' filters (kernel_size/2-1)
        self.conv2=nn.Conv1d(in_channels=1, out_channels=64, kernel_size=2048, stride=1024, padding=1023, bias=True) #Padding for 'same' filters (kernel_size/2-1)
        self.conv3=nn.Conv1d(in_channels=1, out_channels=128, kernel_size=2048, stride=1024, padding=1023, bias=True) #Padding for 'same' filters (kernel_size/2-1)

        #Synthesis filter bank:
        self.synconv1=nn.ConvTranspose1d(in_channels=32, out_channels=1, kernel_size=2048, stride=1024, padding=1023, bias=True)
        self.synconv2=nn.ConvTranspose1d(in_channels=64, out_channels=1, kernel_size=2048, stride=1024, padding=1023, bias=True)
        self.synconv3=nn.ConvTranspose1d(in_channels=128, out_channels=1, kernel_size=2048, stride=1024, padding=1023, bias=True)


    def encoder1(self, x):
        #Analysis:
        x = self.conv1(x)
        y = torch.tanh(x)
        return y

    def encoder2(self, x):
        #Analysis:
        x = self.conv2(x)
        y = torch.tanh(x)
        return y

    def encoder3(self, x):
        #Analysis:
        x = self.conv3(x)
        y = torch.tanh(x)
        return y

    def decoder1(self, y):
        #Synthesis:
        xrek= self.synconv1(y)
        return xrek

    def decoder2(self, y):
        #Synthesis:
        xrek= self.synconv2(y)
        return xrek

    def decoder3(self, y):
        #Synthesis:
        xrek= self.synconv3(y)
        return xrek

    def forward(self, x):
        y=self.encoder1(x)
        y=self.encoder2(y)
        y=self.encoder3(y)
        y=self.decoder3(y)
        y=self.decoder2(y)
        y=self.decoder1(y)


        return y

In [ ]:
# Now you can iterate through the dataloader in your training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Generate Model:")
model = Convautoenc().to(device)
print('Total number of parameters: %i' % (sum(p.numel() for p in model.parameters() if p.requires_grad)))
print("Def. loss function:")
loss_fn = nn.MSELoss()  #MSE

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)#, betas=(0.9, 0.999))

epochs=500
for epoch in range(epochs):
    for batch_idx, (noisy_audio, clean_audio) in enumerate(train_loader):
        noisy_audio = noisy_audio.to(device)
        clean_audio = clean_audio.to(device)

        print("Noisy audio shape",noisy_audio.shape)
        print("Clean audio shape",clean_audio.shape)
        optimizer.zero_grad()
        Ypred = model(noisy_audio[0])
        clean_audio_idx=clean_audio[0]
        outputlen=len(Ypred[0,0,:])
        print("outputlen",outputlen)
        clean_audio_trunc=clean_audio_idx[:,:,:outputlen]
        loss = loss_fn(Ypred, clean_audio_trunc)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch + 1}/{epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

# Don't forget to save your trained model after training
torch.save(model.state_dict(), "your_model_checkpoint.pth")


Generate Model:
Total number of parameters: 917731
Def. loss function:
Shape of noisy audio  torch.Size([2, 96000])
tensor([[-0.0062, -0.0069, -0.0062,  ..., -0.0967, -0.0972, -0.0961],
        [-0.0013, -0.0032, -0.0043,  ..., -0.1013, -0.1042, -0.1070]])
Shape of clean audio  torch.Size([1, 96000])
tensor([[-0.0056, -0.0059, -0.0057,  ..., -0.0994, -0.1017, -0.1036]])
Shape of noisy audio AT  torch.Size([1, 165000])
tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0967, -0.0972, -0.0961]])
Shape of clean audio AT torch.Size([1, 165000])
tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0994, -0.1017, -0.1036]])
Shape of noisy audio  torch.Size([2, 96000])
tensor([[ 0.1639,  0.1706,  0.1713,  ..., -0.0220, -0.0213, -0.0202],
        [ 0.1663,  0.1732,  0.1727,  ..., -0.0166, -0.0163, -0.0161]])
Shape of clean audio  torch.Size([1, 95996])
tensor([[0.1910, 0.1982, 0.1985,  ..., 0.0000, 0.0000, 0.0000]])
Shape of noisy audio AT  torch.Size([1, 165000])
tensor([[ 0.0000,  0.0000,  0.0000,  ..., -

KeyboardInterrupt: ignored

In [ ]:
ww = model.state_dict()   #read obtained weights
noisy_audio_test, ntsamplerate = librosa.load("/content/Noisy/noisy_p232_001_audio_chunk_0.wav", mono=False, sr=None)
noisy_audio_norm_test = noisy_audio_test/np.abs(noisy_audio_test.max())
noisy_audio_norm_test_q=signal2pytorch(noisy_audio_norm_test).to(device)
predictions=model(noisy_audio_norm_test_q).cpu() # Make Predictions based on the obtained weights, on training set
predictions=predictions.detach()
predictions=np.array(predictions)
print(predictions)
xrek=predictions[:,0,:]  #remove unnecessary dimension for playback

[[[-0.02186587 -0.00882358 -0.01862752 ... -0.02882574 -0.02326817
   -0.03375078]]]


In [ ]:
import IPython.display as ipd
display(ipd.Audio(xrek, rate=ntsamplerate));